In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.naive_bayes import MultinomialNB



df = pd.read_pickle("untracked_data/data_prepro_train_01.pkl")

df_test = pd.read_pickle("untracked_data/data_prepro_test_01.pkl")

X_train = df['text']
y_train = df['label']

X_test = df_test['text']
y_test = df_test['label']

In [4]:
vectorizer = CountVectorizer(ngram_range=(1, 1), min_df=5)
X = vectorizer.fit_transform(X_train)

# Train the Multinomial Naive Bayes model
model = MultinomialNB(alpha=0.2)
model.fit(X, y_train)

MultinomialNB(alpha=0.2)

In [10]:
# Build feature dataframe
class_labels = model.classes_
log_probs = pd.DataFrame(model.feature_log_prob_.T, index=features, columns=class_labels)

# Compute log-probability difference between classes
# Positive values → more indicative of first class
# Negative values → more indicative of second class
log_probs['difference'] = log_probs[class_labels[0]] - log_probs[class_labels[1]]

# Sort and extract top 5 terms per class based on discriminative power
top_class0 = log_probs.sort_values(by='difference', ascending=False).head(5)
top_class1 = log_probs.sort_values(by='difference', ascending=True).head(5)

# Make labeled output lists
top_for_label0 = [(term, round(score, 3)) for term, score in zip(top_class0.index, top_class0['difference'])]
top_for_label1 = [(term, round(score, 3)) for term, score in zip(top_class1.index, top_class1['difference'])]

print(f"Top 5 terms for '{class_labels[0]}' with scores:")
print(top_for_label0)

print(f"\nTop 5 terms for '{class_labels[1]}' with scores:")
print(top_for_label1)

Top 5 terms for 'fake' with scores:
[('relax', 4.248), ('originally', 4.096), ('settled', 3.917), ('eggs', 3.814), ('steak', 3.699)]

Top 5 terms for 'real' with scores:
[('priceline', -4.63), ('sofa', -4.278), ('25', -4.278), ('fridge', -4.205), ('stated', -4.126)]
